# Fun with Gaia DR2 Solar System Objects

Woo! Solar System objects in Gaia DR2! With data from SDSS and the MPC

In [1]:
import altair as alt
alt.renderers.enable('notebook')
from altair import datum

import numpy as np
import pandas as pd

alt.data_transformers.enable('json')
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

You should have downloaded the Gaia SSO files in CSV.

Here's the directory I have them in, but you substitute your own:

In [2]:
datadir = "/Users/danielahuppenkothen/work/data/gaiadr2_sso/"

There should be four files with observations, and four files with information about the asteroids themselves. Let's load them all and merge them together.

We're going to read the data into pandas DataFrames.

In [3]:
filenames = ["SsoObservation_-4284702096_-4283326086.csv",
             "SsoObservation_-4284857946_-4284702156.csv",
             "SsoObservation_-4284922936_-4284857966.csv", 
             "SsoObservation_-4284967216_-4284922946.csv"]

In [4]:
for i,f in enumerate(filenames):
    # load the data file
    data = pd.read_csv(datadir+f)
    # find all data points that don't have 
    # a NaN in their g-band magnitude
    idx = data["g_mag"].dropna().index
    data_nonan = data.loc[idx]
    if i == 0:
        data_merged = data_nonan
    else:
        data_merged = pd.concat([data_merged, data_nonan])

In [5]:
len(data_merged)

1030866

In [6]:
data_merged.head()

,solution_id,source_id,observation_id,number_mp,epoch,epoch_err,epoch_utc,ra,dec,ra_error_systematic,...,g_flux,g_flux_error,x_gaia,y_gaia,z_gaia,vx_gaia,vy_gaia,vz_gaia,position_angle_scan,level_of_confidence
5,4427920383700574337,-4284702096,406576829521238051,26520,2132.117075,1.137500e-08,2132.116066,338.836562,-27.330300,8.342880,...,3535.764628,11.155904,0.768081,0.595093,0.257915,-0.011501,0.012148,0.005222,289.327370,0
6,4427920383700574337,-4284702096,406576829521238052,26520,2132.117132,1.137500e-08,2132.116123,338.836575,-27.330279,8.342881,...,3535.764628,11.155904,0.768080,0.595094,0.257915,-0.011501,0.012148,0.005222,289.327470,0
7,4427920383700574337,-4284702096,406576829521238053,26520,2132.117188,1.137500e-08,2132.116179,338.836586,-27.330262,8.342882,...,3535.764628,11.155904,0.768080,0.595094,0.257916,-0.011501,0.012148,0.005222,289.327572,0
8,4427920383700574337,-4284702096,406576829521238054,26520,2132.117244,1.137500e-08,2132.116235,338.836580,-27.330289,8.342881,...,3535.764628,11.155904,0.768079,0.595095,0.257916,-0.011501,0.012148,0.005222,289.327681,0
9,4427920383700574337,-4284702096,406576829521238055,26520,2132.117300,1.137500e-08,2132.116291,338.836592,-27.330271,8.342882,...,3535.764628,11.155904,0.768079,0.595096,0.257916,-0.011501,0.012148,0.005222,289.327782,0


This table contains a row for each *observation* of an asteroid. That is, this is essentially just a giant list of time series, appended to each other. You can get out a single time series (light curve in astronomy-speak) by grouping by a column that will be the same for a given asteroid (but unique compared to other asteroids). I'd suggest using the `number_mp` column. We'll get back to that later!

Let's drop some columns we won't need in the future:

In [7]:
data_merged = data_merged.drop(['dec_error_random',  'dec_error_systematic', 'epoch_utc',
                               'epoch_utc', 'ra_dec_correlation_random',
                                "solution_id", "source_id", "observation_id", "epoch_err", "level_of_confidence",
                               'ra_dec_correlation_systematic', 'ra_error_random', 
                               'ra_error_systematic','vx_gaia', 'vy_gaia',
                               'vz_gaia', 'x_gaia', 'y_gaia', 'z_gaia'], axis=1)

Let's do the same for the source information:

In [8]:
src_filenames = ["SsoSource_-4284702096_-4283326086.csv",
                 "SsoSource_-4284857946_-4284702156.csv",
                 "SsoSource_-4284922936_-4284857966.csv",
                 "SsoSource_-4284967216_-4284922946.csv"]

In [9]:
for i,f in enumerate(src_filenames):
    # load the data file
    source = pd.read_csv(datadir+f)
    # find all data points that don't have 
    # a NaN in their g-band magnitude
    source_nonan = source.dropna()
    if i == 0:
        source_merged = source_nonan
    else:
        source_merged = pd.concat([source_merged, source_nonan])

In [10]:
source.head()

,solution_id,source_id,num_of_obs,number_mp,denomination
0,4166707207534977073,-4284967216,60,8,flora
1,4166707207534977073,-4284967156,79,14,irene
2,4166707207534977073,-4284967126,68,17,thetis
3,4166707207534977073,-4284967106,67,19,fortuna
4,4166707207534977073,-4284967086,85,21,lutetia


We're going to reset the index of our DataFrame to `number_mp`, which is the Minor Planet Center designation. We'll use that to match up these objects with other sources. 

From the source table, we really want the `num_of_obs` (the number of times Gaia observed this asteroid) and the `denomination` (the common name of the asteroid):

In [11]:
source_merged = source_merged.set_index("number_mp")

Let's now join the source table and the data table:

In [12]:
data_merged = data_merged.join(source_merged[["num_of_obs", "denomination"]], how="right", on="number_mp", lsuffix="_obs", rsuffix="_src")

In [13]:
data_merged.head()

,number_mp,epoch,ra,dec,g_mag,g_flux,g_flux_error,position_angle_scan,num_of_obs,denomination
5,26520,2132.117075,338.836562,-27.330300,16.817157,3535.764628,11.155904,289.327370,87,2000 cq75
6,26520,2132.117132,338.836575,-27.330279,16.817157,3535.764628,11.155904,289.327470,87,2000 cq75
7,26520,2132.117188,338.836586,-27.330262,16.817157,3535.764628,11.155904,289.327572,87,2000 cq75
8,26520,2132.117244,338.836580,-27.330289,16.817157,3535.764628,11.155904,289.327681,87,2000 cq75
9,26520,2132.117300,338.836592,-27.330271,16.817157,3535.764628,11.155904,289.327782,87,2000 cq75


Cool! That table now also has the `number_of_obs` and `denomination` columns included!

However, note that the original tables contain a fair number of `NaN` values in the g-band magnitudes, which we've removed. This means that the number of *actual* data points per asteroid is smaller than the `number_of_obs` column suggests. 

Let's calculate the actual number of data points per asteroid:

In [14]:
sourceid_counts = data_merged[["number_mp"]].groupby("number_mp").size().reset_index(name='counts')

In [15]:
sourceid_counts.head()

,number_mp,counts
0,8,46
1,14,67
2,17,44
3,19,56
4,21,45


In [16]:
np.mean(sourceid_counts.counts)

73.12461876728845

In [17]:
np.var(sourceid_counts.counts)

2652.174625493029

Ok, cool. Let's append that column to our DataFrame, too:

In [18]:
data_merged = data_merged.merge(sourceid_counts, on="number_mp")

In [19]:
data_merged.head()

,number_mp,epoch,ra,dec,g_mag,g_flux,g_flux_error,position_angle_scan,num_of_obs,denomination,counts
0,26520,2132.117075,338.836562,-27.330300,16.817157,3535.764628,11.155904,289.327370,87,2000 cq75,32
1,26520,2132.117132,338.836575,-27.330279,16.817157,3535.764628,11.155904,289.327470,87,2000 cq75,32
2,26520,2132.117188,338.836586,-27.330262,16.817157,3535.764628,11.155904,289.327572,87,2000 cq75,32
3,26520,2132.117244,338.836580,-27.330289,16.817157,3535.764628,11.155904,289.327681,87,2000 cq75,32
4,26520,2132.117300,338.836592,-27.330271,16.817157,3535.764628,11.155904,289.327782,87,2000 cq75,32


That information now sits in the `counts` column.

### Adding Minor Planet Center Data

For this part, you'll need the [MPCORB.DAT file](https://minorplanetcenter.net/data), which includes the orbital information for *all* the asteroids in the [Minor Planet Center](https://minorplanetcenter.net) database.

Because this file is huge, we're going to extract just a few columns from it that we're interested in:

In [20]:
mpc_cols = ["number_mp", "abs_mag_h", 
            "inclination", "ecc", 
            "semimajor_axis"]

mpc = pd.read_csv(datadir+"MPCORB.DAT", skiprows=3, sep="\s*", 
                  usecols=[0, 1, 7, 8, 10],
                  names = mpc_cols)

/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/io/parsers.py:2227: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/io/parsers.py:2229: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


In [21]:
mpc.head()

,number_mp,abs_mag_h,inclination,ecc,semimajor_axis
0,00001,3.34,10.59351,0.075535,2.7670463
1,00002,4.13,34.83687,0.230506,2.7727662
2,00003,5.33,12.98973,0.256888,2.6688514
3,00004,3.20,7.14071,0.089068,2.3621036
4,00005,6.85,5.36774,0.191359,2.5734866


There is a lot of data in there that we don't need, because many of the asteroids don't have numerical descriptors. I don't know what those are, but I know I don't have any of those in the Gaia dataset, anyway, so I'm going to exclude them:

In [22]:
mpc = mpc[:99998]

We need to transform the `number_mp` column with the MPC designation into a numerical column so that we can match it up with our Gaia data:

In [23]:
mpc["number_mp"] = pd.to_numeric(mpc["number_mp"])

In [24]:
mpc = mpc.set_index("number_mp")

Now let's extract only the MPC data for the asteroids that we actually have in our Gaia data:

In [25]:
mpc = mpc.loc[data_merged["number_mp"].unique()]

/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


We're going to also calculate some really simplistic (probably wrong) cuts based on eccentricity. This is based on Figure 17 in the [Gaia Solar System Objects paper](https://arxiv.org/abs/1804.09379): 

In [26]:
# initialize empty series for the type
mpc["type"] = np.zeros(len(mpc), dtype=str)
# make semimajor axis into a float array
mpc["semimajor_axis"] = np.array(mpc["semimajor_axis"], dtype=float)
# make cuts and set type to the correct name
mpc.loc[mpc["semimajor_axis"] > 10, "type"] = "Transneptunian Object"
mpc.loc[(mpc["semimajor_axis"] > 4.5) & (mpc["semimajor_axis"] < 10), "type"] = "Jupiter Trojans"
mpc.loc[(mpc["semimajor_axis"] > 1.5) & (mpc["semimajor_axis"] < 4.5), "type"] = "Main-Belt Asteroids"
mpc.loc[(mpc["semimajor_axis"] <= 1.5), "type"] = "Near-Earth Objects"

Let's merge this information with our Gaia data:

In [27]:
data_merged = data_merged.join(mpc, how="right", on="number_mp")

In [28]:
data_merged.head()

,number_mp,epoch,ra,dec,g_mag,g_flux,g_flux_error,position_angle_scan,num_of_obs,denomination,counts,abs_mag_h,inclination,ecc,semimajor_axis,type
0,26520,2132.117075,338.836562,-27.330300,16.817157,3535.764628,11.155904,289.327370,87,2000 cq75,32,13.0,15.23213,0.262485,2.683274,Main-Belt Asteroids
1,26520,2132.117132,338.836575,-27.330279,16.817157,3535.764628,11.155904,289.327470,87,2000 cq75,32,13.0,15.23213,0.262485,2.683274,Main-Belt Asteroids
2,26520,2132.117188,338.836586,-27.330262,16.817157,3535.764628,11.155904,289.327572,87,2000 cq75,32,13.0,15.23213,0.262485,2.683274,Main-Belt Asteroids
3,26520,2132.117244,338.836580,-27.330289,16.817157,3535.764628,11.155904,289.327681,87,2000 cq75,32,13.0,15.23213,0.262485,2.683274,Main-Belt Asteroids
4,26520,2132.117300,338.836592,-27.330271,16.817157,3535.764628,11.155904,289.327782,87,2000 cq75,32,13.0,15.23213,0.262485,2.683274,Main-Belt Asteroids


Did we loose any of our asteroids?

In [29]:
len(data_merged["number_mp"].unique())

14099

Let's store this data in two different json files for plotting: one with all the data points, and one with a random subset of 1400 asteroids:

In [30]:
outfile = "gaiadr2_mpc_full.json"
data_merged.to_json(datadir+outfile, orient="records")

In [31]:
outfile_small = "gaiadr2_mpc_small.json"

# random subset of asteroids
random_sel = np.random.choice(data_merged.number_mp.unique(), 
                              replace=False, size=1500)

# select only asteroids with those chosen MPC designations
data_sel = data_merged.loc[data_merged["number_mp"].isin(random_sel)]

data_sel.to_json(datadir+outfile_small, orient="records")

Let's also save them in csv-files for compatibility:

In [32]:
outfile = "gaiadr2_mpc_full.csv"
data_merged.to_csv(datadir+outfile)

In [33]:
outfile_small = "gaiadr2_mpc_small.csv"

# random subset of asteroids
random_sel = np.random.choice(data_merged.number_mp.unique(), 
                              replace=False, size=1500)

# select only asteroids with those chosen MPC designations
data_sel = data_merged.loc[data_merged["number_mp"].isin(random_sel)]

data_sel.to_csv(datadir+outfile_small)

### SDSS Colours

Gaia has only released g-band magnitudes for the solar system objects. To augment this data, we're going to include measurements from the Sloan Digital Sky Survey, which has observed hundreds of thousands of asteroids. 

You can find the description and the data file for download [here](http://faculty.washington.edu/ivezic/sdssmoc/sdssmoc.html). 

Again, this is a huge file, and we need only a few columns, so let's just download those:

In [34]:
sdss_names = ["u", "g", "r", "i", "z", "number_mp", "sdss_desig", "h-mag"]
sdss_cols = [19, 21, 23, 25, 27, 34, 35, 46]


In [35]:
sdss = pd.read_csv(datadir+"ADR4.dat", sep="\s*",  
                   usecols= sdss_cols,
                   names = sdss_names)

/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/io/parsers.py:2227: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/io/parsers.py:2229: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


In [36]:
sdss.head()

,u,g,r,i,z,number_mp,sdss_desig,h-mag
0,23.51,21.55,21.14,20.87,20.89,0,-,0.00
1,21.81,20.32,19.77,19.56,19.44,62869,2000_UO84,14.60
2,25.02,23.43,21.50,20.68,20.14,0,-,0.00
3,23.28,21.48,20.78,20.75,20.93,0,2004_TG250,16.86
4,19.69,17.91,17.32,17.10,17.05,5212,1989_SS,11.60


Again, we need to transform the `number_mp` column to match it with the column in our Gaia DataFrame of the same name:

In [37]:
sdss["number_mp"] = pd.to_numeric(sdss["number_mp"])
sdss = sdss.set_index("number_mp")

In [38]:
sdss = sdss.loc[data_merged["number_mp"].unique()]

/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


We're going to drop some NaNs:

In [39]:
sdss = sdss.dropna()

In [40]:
len(sdss)

8921

Okay, this means that the combined SDSS+Gaia+MPC data will only contain a subset of ~2/3 of the original Gaia data. Oh well, that's annoying, but not a huge problem.

Before we merge, there are also a few cases where the magnitudes are not NaN, but still not reasonable values. Let's drop those, too:

In [41]:
sdss = sdss.drop(sdss[sdss["u"] == np.max(sdss["u"])].index)
sdss = sdss.drop(sdss[sdss["g"] == np.max(sdss["g"])].index)
sdss = sdss.drop(sdss[sdss["z"] == np.max(sdss["z"])].index)

In [42]:
len(sdss)

8854

Let's also make some colours:

In [43]:
sdss["u_g"] = sdss["u"] - sdss["g"]
sdss["g_r"] = sdss["g"] - sdss["r"]
sdss["r_i"] = sdss["r"] - sdss["i"]
sdss["i_z"] = sdss["i"] - sdss["z"]

And we'll make the colour from [Parker et al, 2008](https://arxiv.org/abs/0807.3762):

In [44]:
sdss["sdss_colour"] = 0.89*(sdss["g"] - sdss["r"]) + \
                      0.45*(sdss["r"] - sdss["i"]) - 0.57 

Let's now join the two data frames:

In [45]:
data_merged = data_merged.join(sdss, how="right", on="number_mp")

In [46]:
data_merged.head()

,number_mp,epoch,ra,dec,g_mag,g_flux,g_flux_error,position_angle_scan,num_of_obs,denomination,...,r,i,z,sdss_desig,h-mag,u_g,g_r,r_i,i_z,sdss_colour
294,26563,2128.614560,86.930188,-0.861110,17.639699,1657.549663,5.989363,226.886862,308,2000 eg39,...,17.63,17.41,17.25,2000_EG39,12.5,1.4,0.55,0.22,0.16,0.0185
295,26563,2128.614616,86.930178,-0.861109,17.639699,1657.549663,5.989363,226.887046,308,2000 eg39,...,17.63,17.41,17.25,2000_EG39,12.5,1.4,0.55,0.22,0.16,0.0185
296,26563,2128.614672,86.930204,-0.861148,17.639699,1657.549663,5.989363,226.887230,308,2000 eg39,...,17.63,17.41,17.25,2000_EG39,12.5,1.4,0.55,0.22,0.16,0.0185
297,26563,2128.614728,86.930197,-0.861149,17.639699,1657.549663,5.989363,226.887414,308,2000 eg39,...,17.63,17.41,17.25,2000_EG39,12.5,1.4,0.55,0.22,0.16,0.0185
298,26563,2128.614785,86.930192,-0.861153,17.639699,1657.549663,5.989363,226.887597,308,2000 eg39,...,17.63,17.41,17.25,2000_EG39,12.5,1.4,0.55,0.22,0.16,0.0185


In [47]:
len(data_merged)

640312

There are still a bunch of measurements in there that aren't great (based on the SDSS u-band magnitude), so let's remove those:

In [48]:
data_good = data_merged[(data_merged["u_g"] < 2.5) & (data_merged["u_g"] > -0.5)]

Let's save the DataFrame to file, too:

In [49]:
outfile = 'gaia_mpc_sdss_full.json'
data_good.to_json(datadir+outfile, orient="records")

In [50]:
outfile = 'gaia_mpc_sdss_full.csv'
data_good.to_csv(datadir+outfile)

And we'll save a subset for easier plotting:

In [51]:
random_sel = np.random.choice(data_good.number_mp.unique(),
                              replace=False, size=1400)

data_sel = data_good.loc[data_good["number_mp"].isin(random_sel)]

In [52]:
outfile = 'gaia_mpc_sdss_small.json'
data_sel.to_json(datadir+outfile, orient="records")

In [53]:
outfile = 'gaia_mpc_sdss_small.csv'
data_sel.to_csv(datadir+outfile)

... and that's it for data preparation! We can now start plotting, but we'll do that elsewhere! 